In [1]:
from src.audioProcessing import splitOnSilence

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
windowsList = []
mypath = './dataset/recordings/stage-1/converted'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for i,file in enumerate(files[:1]):
    fileID = int(re.findall('[0-9]+.',file)[0][:-1])
    species = re.findall('\w+-\w+_',file)[0][:-1]
    chunks = splitOnSilence(f'{mypath}/{file}')
    print(f'Splitting {file}: file {i+1} out of {len(files)}')
    for chunk in chunks:
        # Define windows with overlap:
        windowLen = 1000
        overlap = 500
        if len(chunk) >= 1000:
            for i in range(0,len(chunk),overlap):
                if i+windowLen <= len(chunk):
                    window = chunk[i:i+windowLen]
                    # Get array from each window:
                    window = window.set_frame_rate(48000) # Así puedo cambiar la longitud del array
                    sample = window.get_array_of_samples()
                    # Check if window has a minimum of amplitude... (para intentar quitar las ventanas donde no hay pájaro...)
                    if np.max(sample) > 1500:
                        # Fourier:
                        fft_mod = np.abs(fft(sample,512))
                        fft_mod = fft_mod[0:len(fft_mod)//2]
                        # Array of dictionaries:
                        windowsList.append({'class':species,'id':fileID,'sound':sample,'fourier':fft_mod,'mfcc':0})

Splitting Corvus-corax_168257.mp3: file 1 out of 464


In [28]:
DF = pd.DataFrame(windowsList)

In [29]:
DF.head()

,class,fourier,id,mfcc,sound
0,Corvus-corax,"[4056.0, 5429.635666221412, 8618.846488201112,...",168257,0,"[272, 330, 380, 420, 445, 455, 453, 442, 420, ..."
1,Corvus-corax,"[7170.0, 8602.57465157886, 16452.318372583995,...",168257,0,"[-5, -6, 1, 18, 41, 67, 94, 118, 134, 139, 135..."
2,Corvus-corax,"[44345.0, 49459.83958723293, 75450.42234918126...",168257,0,"[1714, 1632, 1497, 1313, 1087, 835, 575, 317, ..."
3,Corvus-corax,"[5328.0, 6216.437657063371, 12172.538232679979...",168257,0,"[53, 105, 147, 186, 224, 259, 286, 305, 318, 3..."
4,Corvus-corax,"[4398.0, 4344.279161172816, 4993.564517380789,...",168257,0,"[-677, -537, -396, -262, -135, -15, 104, 224, ..."


In [7]:
len(DF.sound[0])

47999

In [9]:
from scipy.fftpack import fft

In [22]:
# %matplotlib inline
fft_mod = np.abs(fft(sample,512))
fft_mod = fft_mod[0:len(fft_mod)//2]
# plt.stem(fft_mod, use_line_collection=True)
len(fft_mod)